In [ ]:
# Start writing code here...

# JAX

Jax est une bibliothèque Python spécialisée dans le calcul différentiel (localement) distribué. Jax utilise XLA pour compiler le code Python sur des processeurs spécialisés (GPU, TPU).

## 1. Quickstart

https://jax.readthedocs.io/en/latest/notebooks/quickstart.html#



In [ ]:
from jax import random
key = random.PRNGKey(0)
x = random.normal(key, (10,))
print(x)

[-0.372111    0.2642311  -0.18252774 -0.7368198  -0.44030386 -0.15214427
 -0.6713536  -0.59086424  0.73168874  0.56730247]


In [ ]:
import jax.numpy as jnp
size = 3000
x = random.normal(key, (size, size), dtype=jnp.float32)
# JAX uses asynchronous execution by default
%timeit jnp.dot(x, x.T).block_until_ready() 

657 ms ± 32.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# JAX NumPy functions work on regular NumPy arrays.
import numpy as np
x = np.random.normal(size=(size, size)).astype(np.float32)
%timeit jnp.dot(x, x.T).block_until_ready()

601 ms ± 6.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 1.1 Compilation juste-à-temps avec `jit()`

Par défaut, chaque instruction Jax est envoyée, ligne à ligne, au processeur dédié (GPU, TPU). Pourtant, une succession d'instructions a de grande chance d'être plus rapidement exécutée que des instructions séparées. POur cela on utilise `jit()`.

In [ ]:
def selu(x, alpha=1.67, lmbda=1.05):
  return lmbda * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)

x = random.normal(key, (1000000,))
%timeit selu(x).block_until_ready()

2.62 ms ± 295 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
from jax import jit
selu_jit = jit(selu)
%timeit selu_jit(x).block_until_ready()

666 µs ± 51.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### 1.2 Différenciation automatique avec `grad()`

In [ ]:
def sum_logistic(x):
  return jnp.sum(1.0 / (1.0 + jnp.exp(-x)))

x_small = jnp.arange(3.)

from jax import grad
derivative_fn = grad(sum_logistic)
print(derivative_fn(x_small))

[0.25       0.19661197 0.10499357]


... à comparer avec une différenciation numérique:

In [ ]:
def first_finite_differences(f, x):
  eps = 1e-3 # epsilon machine
  return jnp.array([(f(x + eps * v) - f(x - eps * v)) / (2 * eps)
                   for v in jnp.eye(len(x))])


print(first_finite_differences(sum_logistic, x_small))

[0.24998187 0.1965761  0.10502338]


In [ ]:
grad(grad(sum_logistic))(1.) # possible to chain grad()

DeviceArray(-0.09085775, dtype=float32, weak_type=True)

In [ ]:
# possible to combine with jit()
jit(grad(grad(sum_logistic)))(1.)

DeviceArray(-0.09085775, dtype=float32, weak_type=True)

### 1.3 Vectorisation à la volée avec `vmap()`

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f3692ed-5f27-49a4-899a-82a03e72232c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>